# Diving into French Presidential Discourses
## Jean-Luc Mélenchon (JLM)

In [1]:
import pandas as pd
from pdfminer.high_level import extract_text
from gensim.test.utils import datapath
from gensim import utils
import spacy
from collections import Counter
import unidecode
import gensim.models

In [2]:
## create a .csv of JLM's Twitter activity since 27 Jan 2022
!minet tw scrape tweets "(from:JLMelenchon) until:2022-03-27 since:2022-01-27" > tweets_JLM.csv

## convert the .csv file in a data frame using pandas
df_tw_JLM = pd.read_csv("./tweets_JLM.csv")

## create a list of tweets selecting the 'text' column of the data frame
list_tw_JLM = df_tw_JLM['text'].values.tolist()
len(list_tw_JLM)

Searching for "(from:JLMelenchon) until:2022-03-27 since:2022-01-27"


1495

In [3]:
## retrieve JLM's affiliates1 Twitter activity
!minet tw scrape tweets "(from:AQuatennens OR from:JulieGarnierFI OR from:mbompard OR from:ALeaument OR from:Clemence_Guette OR from:Francois_Ruffin OR from:MathildePanot OR from:alexiscorbiere OR from:ClementVerde OR from:Aurelien_Le_Coq OR from:Deputee_Obono OR from:ericcoquerel OR from:Clem_Autain OR from:PrudhommeLoic OR from:BenedictTaurine) until:2022-03-27 since:2022-01-27" > tw_JLM_aff_all.csv

## convert JLM's affiliates' tweets in a list
df_tw_JLM_aff_all = pd.read_csv("tw_JLM_aff_all.csv")
list_tw_JLM_aff_all = df_tw_JLM_aff_all['text'].values.tolist()
print(list_tw_JLM_aff_all[0])
print(len(list_tw_JLM_aff_all))

Searching for "(from:AQuatennens OR from:JulieGarnierFI OR from:mbompard OR from:ALeaument OR from:Clemence_Guette OR from:Francois_Ruffin OR from:MathildePanot OR from:alexiscorbiere OR from:ClementVerde OR from:Aurelien_Le_Coq OR from:Deputee_Obono OR from:ericcoquerel OR from:Clem_Autain OR from:PrudhommeLoic OR from:BenedictTaurine) until:2022-03-27 since:2022-01-27"
Notre système de santé a longtemps été admiré. Ce n'est plus le cas. Ce gouvernement a fermé 17 000 lits. 5 900 pendant la crise sanitaire. Ma collègue @CarolineFiat54 a fait la tournée des Ehpad et a écrit un rapport qui établit les besoins à 300 000 personnels. #OEED https://twitter.com/AQuatennens/status/1507868500802654225/video/1
3746


In [4]:
## convert .csv affiliates' tweets in a list
df_tw_JLM_aff_all = pd.read_csv("tw_JLM_aff_all.csv")
list_tw_JLM_aff_all = df_tw_JLM_aff_all['text'].values.tolist()
print(list_tw_JLM_aff_all[0])
print(len(list_tw_JLM_aff_all))

Notre système de santé a longtemps été admiré. Ce n'est plus le cas. Ce gouvernement a fermé 17 000 lits. 5 900 pendant la crise sanitaire. Ma collègue @CarolineFiat54 a fait la tournée des Ehpad et a écrit un rapport qui établit les besoins à 300 000 personnels. #OEED https://twitter.com/AQuatennens/status/1507868500802654225/video/1
3746


In [5]:
## merge JLM's and JLM's affiliates lists
list_tw_JLM_all = list_tw_JLM + list_tw_JLM_aff_all

In [6]:
## i retrieve a string from the pdf of JLM's manifesto using extract_text of the pdfminer package
## the cleaning process is specific for this manifesto and it depends on the output of extract_text
manif_JLM = extract_text('/Users/simonemariaparazzoli/Documents/Università/Sciences Po/Diving into public digital spaces/research/manifesto_melenchon.pdf')
manif_clean_JLM = manif_JLM.replace(' .','')
manif_clean_JLM = manif_clean_JLM.replace('   ','')
manif_clean_JLM = manif_clean_JLM.replace('\n\n','---')
manif_clean_JLM = manif_clean_JLM.replace('\n','')
manif_clean_JLM = manif_clean_JLM.replace('\u202f',' ')
manif_clean_JLM = manif_clean_JLM.replace('\x0c',' ')
#print(repr(manif_clean_JLM))

## convert the string of the manifesto into a list
list_manif_JLM = manif_clean_JLM.split("---")
list_manif_JLM = [s for s in list_manif_JLM if len(s)>20]
len(list_manif_JLM)
#print(list_manif_EM)

952

In [7]:
## merge the two lists of tweets and of the manifesto 
list_JLM = list_tw_JLM_all + list_manif_JLM
len(list_JLM)

6193

In [8]:
## load a spacy model to retrieve stop words
nlp = spacy.load("fr_core_news_sm")
stop_words_fr = nlp.Defaults.stop_words
#new_sw = ["avec","la","les","le","pour","un","une","nous","ete","et","je"]
#stop_words_fr.add(new_sw)

## clean the list of tweets and manifesto to get rid of useless words and make the list content homogeneous
list_JLM_clean = []
for i in list_JLM:
    doc = nlp(i)
    tokens = [unidecode.unidecode(token.text).lower()for token in doc 
              if ( token.text not in stop_words_fr and
                  len(token.text)>1 and
                  token.like_url == False )]
    tokens_joined = ' '.join(tokens)
    list_JLM_clean.append(tokens_joined)

In [9]:
## test the output of the cleaning process
print(list_JLM[2401])
print("---")
print(list_JLM_clean[2401])

Aussi, je vous demande à nouveau, Madame la Ministre : que contient ce rapport ? Vous avez l’obligation légale de nous le remettre, ce qui aurait déjà dû être fait depuis plus années. Je passerai à votre ministère, ce mardi après-midi, pour récupérer ce rapport.
---
aussi demande madame ministre contient rapport vous avez obligation legale remettre du annees je passerai ministere mardi apres-midi recuperer rapport


In [14]:
## prepare the corpus as a class
class MyCorpus_JLM:

    def __iter__(self):
        for i in list_JLM_clean:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(i,min_len=3)

In [43]:
## train the word embeddings model_JLM
sentences = MyCorpus_JLM()
model_JLM = gensim.models.Word2Vec(sentences=sentences, min_count=10, vector_size=300, epochs=100)

In [44]:
## transform the corpus list (that is made of tweets and sentences from the manifesto)
## in a list containing all the words of the corpus as elements of the list
words_JLM = []

for i in list_JLM_clean:
    i_split = i.split(' ') #transform the i document into a list (split at blank space)
    words_JLM.extend(i_split)

## clean the list of tokens
words_JLM_clean = [x for x in words_JLM 
                   if x not in stop_words_fr
                   if x != "\n\n"
                   if len(x)>1]

In [45]:
## find the 30 most common words using Counter
words_freq_JLM = Counter(words_JLM_clean)
common_words_JLM = words_freq_JLM.most_common(100)
print(common_words_JLM)

[('@jlmelenchon', 869), ('france', 687), ('melenchon', 657), ('macron', 614), ('faire', 546), ('faut', 525), ('ans', 474), ('melenchon2022', 468), ('jean-luc', 412), ('contre', 396), ('tour', 372), ('programme', 364), ('francais', 318), ('monde', 308), ('republique', 307), ('pays', 289), ('personnes', 288), ('soir', 277), ('retraite', 275), ('politique', 267), ('guerre', 266), ('prix', 264), ('ukraine', 261), ('meeting', 251), ('elysee2022', 251), ('loi', 251), ('-vous', 248), ('campagne', 242), ('ete', 240), ('peuple', 237), ('...', 237), ('travail', 236), ('second', 234), ('millions', 225), ('ecologique', 221), ('sommes', 221), ('face', 218), ('bien', 217), ('proposition', 214), ('place', 212), ('10', 210), ("aujourd'hui", 209), ('direct', 208), ('candidat', 207), ('60', 199), ('droit', 198), ('propose', 197), ('rendez', 196), ('poutine', 195), ('temps', 195), ('populaire', 194), ('presidentielle', 192), ('etat', 191), ('non', 190), ('paix', 190), ('melenchontf1', 189), ('soutien', 1

In [46]:
## first attempt with the most_similar function on our corpus using our model_JLM
result = model_JLM.wv.most_similar(positive=['vie'], topn=30)
print(result)

[('population', 0.24711108207702637), ('atteindre', 0.24432621896266937), ('seuil', 0.24298720061779022), ('anciens', 0.23981203138828278), ('htoucel', 0.23491954803466797), ('ces', 0.23244190216064453), ('pesticides', 0.23099255561828613), ('insupportable', 0.22286000847816467), ('vivre', 0.22253501415252686), ('agir', 0.22162723541259766), ('puisse', 0.22026650607585907), ('vit', 0.21682676672935486), ('million', 0.2108323872089386), ('eleve', 0.210114523768425), ('cout', 0.2094697803258896), ('misere', 0.20871585607528687), ('permettra', 0.20846790075302124), ('entre', 0.20712895691394806), ('hebergement', 0.20459675788879395), ('possedent', 0.20456978678703308), ('approche', 0.2028636783361435), ('sante', 0.20239152014255524), ('milliardaires', 0.20228594541549683), ('jeune', 0.20200707018375397), ('glyphosate', 0.20032401382923126), ('dessus', 0.19987136125564575), ('atteint', 0.19941624999046326), ('falloir', 0.19894060492515564), ('riche', 0.19812563061714172), ('metres', 0.1974

In [47]:
print(len(list_tw_JLM_all),',',len(list_JLM_clean))

5241 , 6193


In [48]:
result = model_JLM.wv.most_similar(positive=['france'], topn=20)
print(result)

[('usa', 0.24648790061473846), ('alignee', 0.22727586328983307), ('frontiere', 0.22709207236766815), ('train', 0.21569570899009705), ('souveraine', 0.2059156447649002), ('defend', 0.20443682372570038), ('course', 0.20095115900039673), ('affirmer', 0.19882535934448242), ('estime', 0.19413191080093384), ('melenchonlyon', 0.19346848130226135), ('refugies', 0.1887848973274231), ('directan', 0.18602809309959412), ('puissance', 0.18359069526195526), ('arrivee', 0.1801304966211319), ('crois', 0.1790710836648941), ('planete', 0.17846451699733734), ('patrie', 0.17605209350585938), ('role', 0.17524738609790802), ('rappelle', 0.174143984913826), ('satellites', 0.17340178787708282)]


In [49]:
result = model_JLM.wv.most_similar(positive=['etat'], topn=20)
print(result)

[('cabinets', 0.337420254945755), ('fiscaux', 0.24584028124809265), ('respecter', 0.23675207793712616), ('deficit', 0.2360282838344574), ('publics', 0.22046513855457306), ('privatisation', 0.22023651003837585), ('autoritaire', 0.21940897405147552), ('societes', 0.21389998495578766), ('etats', 0.21115559339523315), ('privees', 0.20789475739002228), ('principes', 0.20461809635162354), ('redonner', 0.20208920538425446), ('pleine', 0.19745908677577972), ('coup', 0.1917140930891037), ('rompre', 0.19026558101177216), ('publiques', 0.19006794691085815), ('faite', 0.18990181386470795), ('remettre', 0.18950901925563812), ('centre', 0.18949724733829498), ('ces', 0.18938839435577393)]


In [50]:
result = model_JLM.wv.most_similar(positive=['souverainete'], topn=20)
print(result)

[('institutions', 0.42223259806632996), ('rester', 0.4055566191673279), ('independance', 0.36785852909088135), ('banque', 0.35217034816741943), ('permettant', 0.3502298891544342), ('developpement', 0.34550273418426514), ('spatiale', 0.3450522720813751), ('donnees', 0.32997724413871765), ('developper', 0.3263959586620331), ('instaurer', 0.317801296710968), ('individuelle', 0.316335529088974), ('concurrence', 0.315285325050354), ('permanente', 0.31169506907463074), ('agricole', 0.30720826983451843), ('energetique', 0.3053456246852875), ('modele', 0.30466410517692566), ('stopper', 0.30324310064315796), ('forets', 0.3007609248161316), ('alliances', 0.2960357069969177), ('defends', 0.295767605304718)]


In [51]:
result = model_JLM.wv.most_similar(positive=['president'], topn=20)
print(result)

[('invasion', 0.2692466676235199), ('disait', 0.26511961221694946), ('position', 0.2618979215621948), ('rappelle', 0.2608511447906494), ('carolinefiat', 0.24939249455928802), ('vive', 0.24796058237552643), ('emmanuel', 0.24518802762031555), ('defend', 0.24350284039974213), ('elu', 0.23988281190395355), ('parraine', 0.23335646092891693), ('pret', 0.22913840413093567), ('sortie', 0.22294801473617554), ('peuple', 0.21927402913570404), ('sort', 0.21802279353141785), ('entree', 0.21324467658996582), ('reacteurs', 0.2097339630126953), ('geoffroyrdb', 0.19734756648540497), ('pouvoirs', 0.19707337021827698), ('deputeratenon', 0.19591893255710602), ('honneur', 0.195525661110878)]


In [52]:
result = model_JLM.wv.most_similar(positive=['politique'], topn=20)
print(result)

[('democratique', 0.2518428564071655), ('face', 0.2417043149471283), ('realiste', 0.2353464961051941), ('importante', 0.22488322854042053), ('bataille', 0.2117290049791336), ('monsieur', 0.19813257455825806), ('vraie', 0.19810932874679565), ('bilan', 0.1930806040763855), ('rupture', 0.19190536439418793), ('democratie', 0.19079701602458954), ('simple', 0.18830181658267975), ('modele', 0.1837032437324524), ('achat', 0.1836300492286682), ('volonte', 0.1832476109266281), ('transition', 0.18267914652824402), ('gouverner', 0.17997989058494568), ('fond', 0.17821185290813446), ('inegalites', 0.17642375826835632), ('page', 0.17616234719753265), ('permanente', 0.17261004447937012)]


In [53]:
result = model_JLM.wv.most_similar(positive=['droit'], topn=20)
print(result)

[('effectif', 0.3464275896549225), ('droits', 0.2942284345626831), ('detache', 0.27648890018463135), ('etablir', 0.2698085606098175), ('controle', 0.2540951073169708), ('communs', 0.2529160678386688), ('israel', 0.2500777840614319), ('regime', 0.2464696168899536), ('fondamental', 0.24551831185817719), ('laique', 0.24517609179019928), ('constitution', 0.24297499656677246), ('revoquer', 0.23858441412448883), ('gratuite', 0.23839588463306427), ('application', 0.23329417407512665), ('laisse', 0.22434665262699127), ('principe', 0.2232731431722641), ('reconnaissance', 0.21577279269695282), ('precaires', 0.21361735463142395), ('dignite', 0.20952026546001434), ('juge', 0.20882217586040497)]


In [54]:
result = model_JLM.wv.most_similar(positive=['entreprise'], topn=20)
print(result)

[('reconnaitre', 0.3579872250556946), ('asile', 0.34426239132881165), ('code', 0.3400542438030243), ('numero', 0.3377625048160553), ('actionnaires', 0.3311500549316406), ('contrat', 0.33029115200042725), ('favoriser', 0.32778400182724), ('but', 0.32532110810279846), ('societe', 0.31941163539886475), ('casse', 0.3073563575744629), ('benefice', 0.3051731586456299), ('echelle', 0.30310606956481934), ('privatisation', 0.30305981636047363), ('multinationales', 0.30074867606163025), ('meme', 0.30007627606391907), ('pourront', 0.2986007034778595), ('interets', 0.29708999395370483), ('salaries', 0.2932092249393463), ('pouvoirs', 0.28727346658706665), ('empeche', 0.2842700779438019)]


In [55]:
result = model_JLM.wv.most_similar(positive=['economie'], topn=20)
print(result)

[('finance', 0.39257359504699707), ('inverse', 0.3521369993686676), ('besoins', 0.3282887637615204), ('ecologique', 0.3265765309333801), ('cree', 0.3254067599773407), ('disant', 0.30997875332832336), ('financiers', 0.29931241273880005), ('protectionnisme', 0.29474928975105286), ('syndicats', 0.28555193543434143), ('consommation', 0.28499940037727356), ('communes', 0.2807953953742981), ('developper', 0.27926018834114075), ('solidaire', 0.2727420926094055), ('somme', 0.26867911219596863), ('records', 0.2648579180240631), ('eradiquer', 0.2623402774333954), ('entreprise', 0.26200157403945923), ('institutions', 0.2433778941631317), ('harmonie', 0.24250005185604095), ('terre', 0.24234463274478912)]


In [56]:
result = model_JLM.wv.most_similar(positive=['emploi'], topn=20)
print(result)

[('taux', 0.3688880503177643), ('chomeurs', 0.3651057779788971), ('parvenir', 0.3477570712566376), ('pole', 0.34425556659698486), ('emplois', 0.3272612988948822), ('inscrits', 0.3238644599914551), ('chomage', 0.3213678300380707), ('plein', 0.30394482612609863), ('precaires', 0.29649266600608826), ('plans', 0.2886260449886322), ('contrats', 0.28010618686676025), ('precarite', 0.2735254168510437), ('pauvrete', 0.27324673533439636), ('jeunesse', 0.26897141337394714), ('baisse', 0.268794447183609), ('retraite', 0.26690930128097534), ('travaillent', 0.26564115285873413), ('atteindre', 0.25847938656806946), ('rsa', 0.2511855363845825), ('cree', 0.24956907331943512)]


In [57]:
result = model_JLM.wv.most_similar(positive=['travail'], topn=20)
print(result)

[('produit', 0.31458398699760437), ('site', 0.2918703556060791), ('capital', 0.2905762791633606), ('travaillent', 0.2882193326950073), ('respirer', 0.26950880885124207), ('diminuer', 0.2588966488838196), ('bataille', 0.2515246272087097), ('attendant', 0.24488291144371033), ('reduction', 0.23190408945083618), ('aide', 0.230624720454216), ('version', 0.22862190008163452), ('rsa', 0.22672909498214722), ('groupe', 0.22203096747398376), ('ministere', 0.21572916209697723), ('mauvais', 0.21548084914684296), ('travailleurs', 0.2147679328918457), ('examen', 0.21448756754398346), ('amendements', 0.21387772262096405), ('animale', 0.213406041264534), ('financier', 0.20901213586330414)]


In [58]:
result = model_JLM.wv.most_similar(positive=['numerique'], topn=20)
print(result)

[('mediterranee', 0.36114734411239624), ('donnees', 0.3601059913635254), ('partout', 0.34807854890823364), ('nation', 0.33281245827674866), ('affirmer', 0.3315914273262024), ('pratiques', 0.3303614556789398), ('satellites', 0.3202833831310272), ('maritime', 0.31597861647605896), ('cooperations', 0.314937025308609), ('lcp', 0.30645954608917236), ('independance', 0.3016452193260193), ('stopper', 0.2998850643634796), ('banque', 0.2997155487537384), ('humains', 0.2995586097240448), ('assurer', 0.29725760221481323), ('sein', 0.29564976692199707), ('mission', 0.2943326532840729), ('commune', 0.29432234168052673), ('retrouvons', 0.29406410455703735), ('information', 0.2846916913986206)]


In [59]:
result = model_JLM.wv.most_similar(positive=['donnees'], topn=20)
print(result)

[('hebergement', 0.5018681287765503), ('permettant', 0.4758477210998535), ('utilisation', 0.4530894160270691), ('garantir', 0.4184090793132782), ('renforcer', 0.41716113686561584), ('assurer', 0.4144517779350281), ('lieux', 0.41357430815696716), ('activites', 0.4039919972419739), ('domaine', 0.3927300274372101), ('biodiversite', 0.3858721852302551), ('projets', 0.38574281334877014), ('marges', 0.3735058903694153), ('creation', 0.3726143538951874), ('missions', 0.37140560150146484), ('etablir', 0.3700575530529022), ('numerique', 0.3601060211658478), ('mission', 0.3498099446296692), ('etc', 0.3441031873226166), ('stopper', 0.33779966831207275), ('prives', 0.33506324887275696)]


In [60]:
result = model_JLM.wv.most_similar(positive=['monde'], topn=20)
print(result)

[('magnifique', 0.23453359305858612), ('fonctionnement', 0.233946293592453), ('espoir', 0.2308783084154129), ('changer', 0.2284015566110611), ('humain', 0.22374588251113892), ('melenchontours', 0.21561703085899353), ('maltraitance', 0.21223099529743195), ('fort', 0.2118031233549118), ('vies', 0.21171997487545013), ('prets', 0.21109451353549957), ('rupture', 0.20902320742607117), ('tourner', 0.1959989070892334), ('pouvons', 0.19535186886787415), ('stop', 0.19445420801639557), ('votez', 0.19227147102355957), ('alignee', 0.19189338386058807), ('sujets', 0.190670445561409), ('planification', 0.18745827674865723), ('etres', 0.18692737817764282), ('respirer', 0.1862351894378662)]


In [61]:
result = model_JLM.wv.most_similar(positive=['realite'], topn=20)
print(result)

[('fondamental', 0.36452025175094604), ('economistes', 0.332093745470047), ('animaux', 0.31639564037323), ('etres', 0.28714418411254883), ('libertes', 0.2865492105484009), ('serieusement', 0.25505566596984863), ('humains', 0.25502416491508484), ('citoyens', 0.25197142362594604), ('souffrent', 0.24789005517959595), ('lcp', 0.24728433787822723), ('protection', 0.2466052770614624), ('recours', 0.24358029663562775), ('genre', 0.2399033010005951), ('garantir', 0.2366165816783905), ('bien', 0.23658762872219086), ('asile', 0.23570190370082855), ('inscrits', 0.23298722505569458), ('tes', 0.23140870034694672), ('permettra', 0.22876285016536713), ('garantie', 0.228196918964386)]


In [62]:
result = model_JLM.wv.most_similar(positive=['verite'], topn=20)
print(result)

[('mensonge', 0.39767104387283325), ('faureolivier', 0.3664972484111786), ('allemagne', 0.35784828662872314), ('esclavage', 0.3457072377204895), ('article', 0.3251926004886627), ('faux', 0.3162020742893219), ('zelensky', 0.31395483016967773), ('moscou', 0.31172454357147217), ('libertes', 0.30562272667884827), ('constitution', 0.3019994795322418), ('policiers', 0.3008458912372589), ('sort', 0.29859650135040283), ('accepte', 0.29368215799331665), ('armement', 0.2928525507450104), ('septembre', 0.2913445234298706), ('repete', 0.29126039147377014), ('positions', 0.2779848277568817), ('neutre', 0.27734577655792236), ('jaures', 0.2754230201244354), ('secret', 0.27156344056129456)]


In [63]:
result = model_JLM.wv.most_similar(positive=['societe'], topn=20)
print(result)

[('entreprise', 0.3194116950035095), ('harmonie', 0.3160158097743988), ('empeche', 0.3135811686515808), ('carbone', 0.2919323146343231), ('individuelle', 0.28225177526474), ('melenchonclimat', 0.28046929836273193), ('largement', 0.2677189111709595), ('necessaire', 0.26722514629364014), ('regles', 0.2648303508758545), ('etres', 0.26275724172592163), ('davantage', 0.2565653324127197), ('avenirencommun', 0.2529608905315399), ('impose', 0.25204774737358093), ('societes', 0.2502685487270355), ('humain', 0.2489580363035202), ('recettes', 0.2483254224061966), ('travaille', 0.243994802236557), ('programmes', 0.23962144553661346), ('objectif', 0.23918533325195312), ('developper', 0.2373615950345993)]


In [64]:
result = model_JLM.wv.most_similar(positive=['pouvoir'], topn=20)
print(result)

[('terme', 0.2998857796192169), ('achat', 0.29662567377090454), ('depenses', 0.2769147455692291), ('recettes', 0.2664085924625397), ('gilets', 0.24433265626430511), ('reelle', 0.22543691098690033), ('jaunes', 0.22389496862888336), ('institutions', 0.21973344683647156), ('intentions', 0.21495315432548523), ('ric', 0.21348655223846436), ('elus', 0.21198269724845886), ('droits', 0.2064894586801529), ('cadeaux', 0.20228835940361023), ('monsieur', 0.202225461602211), ('permettra', 0.20172280073165894), ('permettre', 0.20108242332935333), ('augmente', 0.1948612481355667), ('clairement', 0.19428621232509613), ('baisse', 0.19334247708320618), ('oligarques', 0.19200780987739563)]


In [65]:
result = model_JLM.wv.most_similar(positive=['avenir'], topn=20)
print(result)

[('enthoven_r', 0.2616974711418152), ('associations', 0.25057360529899597), ('discussion', 0.23746195435523987), ('bien', 0.2337382435798645), ('transports', 0.22612276673316956), ('avenirencommun', 0.22544428706169128), ('defi', 0.22155486047267914), ('jeunesse', 0.21360045671463013), ('taubira', 0.21308758854866028), ('remettre', 0.21091774106025696), ('permettant', 0.21070879697799683), ('solide', 0.20611554384231567), ('credible', 0.204568549990654), ('formation', 0.20402808487415314), ('pauvrete', 0.20185983180999756), ('reconstruire', 0.19992555677890778), ('base', 0.19976076483726501), ('groupes', 0.1982080042362213), ('legislatives', 0.19645598530769348), ('sortir', 0.19566918909549713)]


In [66]:
result = model_JLM.wv.most_similar(positive=['histoire'], topn=20)
print(result)

[('tourner', 0.2920237183570862), ('famille', 0.28008586168289185), ('capitalisme', 0.2585248649120331), ('transformer', 0.2582710087299347), ('zemmoureric', 0.25675684213638306), ('pire', 0.25560325384140015), ('capacite', 0.24940232932567596), ('envoyer', 0.2456735074520111), ('vit', 0.2424473613500595), ('satellites', 0.23918627202510834), ('republicaine', 0.23733407258987427), ('conscience', 0.2370367795228958), ('villepin', 0.22930188477039337), ('salle', 0.22850820422172546), ('peuple', 0.2245374619960785), ('changer', 0.22444447875022888), ('monte', 0.22275008261203766), ('melenchonlemans', 0.2181875854730606), ('aime', 0.2153630256652832), ('petrole', 0.21357643604278564)]


In [67]:
result = model_JLM.wv.most_similar(positive=['contre'], topn=20)
print(result)

[('outil', 0.3235023319721222), ('milliard', 0.26949477195739746), ('fraude', 0.26852700114250183), ('appel', 0.2584410607814789), ('evasion', 0.2511170506477356), ('menee', 0.2498820424079895), ('amendements', 0.249657541513443), ('mene', 0.24696941673755646), ('sexuelles', 0.24224060773849487), ('terrorisme', 0.2418043464422226), ('actuel', 0.23445314168930054), ('meetinganglet', 0.23166736960411072), ('reprendre', 0.22983859479427338), ('zones', 0.22892270982265472), ('refuser', 0.22632868587970734), ('gestion', 0.22495585680007935), ('economiques', 0.22421573102474213), ('courage', 0.22347965836524963), ('salaries', 0.22080062329769135), ('promesse', 0.21991871297359467)]


In [68]:
result = model_JLM.wv.most_similar(positive=['faut'], topn=20)
print(result)

[('devons', 0.32481908798217773), ('priorite', 0.3203314244747162), ('dechets', 0.2834846079349518), ('falloir', 0.27971428632736206), ('paysans', 0.27002355456352234), ('baisser', 0.2693289816379547), ('gris', 0.2661232054233551), ('traiter', 0.2650889754295349), ('imposer', 0.2570405602455139), ('veux', 0.25492343306541443), ('certains', 0.24793541431427002), ('humainement', 0.24518556892871857), ('souffrance', 0.24492228031158447), ('choses', 0.2410561591386795), ('causes', 0.24078181385993958), ('problemes', 0.2392433136701584), ('continuer', 0.23839940130710602), ('profit', 0.23636963963508606), ('production', 0.2348526418209076), ('cas', 0.23435978591442108)]


In [69]:
result = model_JLM.wv.most_similar(positive=['crise'], topn=20)
print(result)

[('consequences', 0.4037177860736847), ('covid', 0.36591705679893494), ('sanitaire', 0.36448371410369873), ('profiteurs', 0.3608472943305969), ('pandemie', 0.338763564825058), ('changement', 0.3375840485095978), ('urgence', 0.32977795600891113), ('chapitre', 0.3144138753414154), ('societes', 0.31244954466819763), ('gestion', 0.3038826584815979), ('fortunes', 0.3011041283607483), ('milliardaires', 0.2999589145183563), ('masques', 0.27966031432151794), ('comment', 0.2746522128582001), ('reconstruire', 0.26449212431907654), ('sante', 0.2590735852718353), ('etc', 0.2545175850391388), ('vit', 0.2460712045431137), ('europeens', 0.24079933762550354), ('catastrophe', 0.2405877262353897)]
